<a href="https://colab.research.google.com/drive/1c9zb9i83WUuiOZmohDoaff0GF6X-RwPT?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Deeply explore RAG working, how does sub-query work?
- Deploy Ragatune

- Fundamental problem - How to read multiple policy documents, contradicting previous ones. How to find latest policy terms.
- Long term Problem Statement - There is a policy, and there is an evidence document (a document to prove that we compliant with policy, and auditor needs to check). How much evidence document matches with policies?
29th Mar (done):
- What is default answer in case of contradictory information.
- In instruct mode, give some email content and ask if it is according to policy

### Preparation

In [ ]:
!pip install -q -U llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install -q -U llama-index-question-gen-guidance
!pip install -q -U llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.2 MB/s eta 0:00:00


In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your api key here"

import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [ ]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

### Download Data

In [ ]:
import os

# Create the 'data/' directory if it doesn't exist
os.makedirs('data/', exist_ok=True)

from google.colab import files

uploaded = files.upload() #here upload the three policy file created for testing

# Move the uploaded files to the 'data/' directory
for filename in uploaded.keys():
    os.rename(filename, os.path.join('data/', filename))

Saving policy-circular-2.pdf to policy-circular-2.pdf
Saving policy-circular-1.pdf to policy-circular-1.pdf
Saving policy.pdf to policy.pdf


In [ ]:
# load data
policy_data = SimpleDirectoryReader(input_files=['./data/policy.pdf']).load_data()
policy_circular_1 = SimpleDirectoryReader(input_files=["./data/policy-circular-1.pdf"]).load_data()
policy_circular_2 = SimpleDirectoryReader(input_files=["./data/policy-circular-2.pdf"]).load_data()

print(policy_data)

# build index and query engine
policy_data_engine = VectorStoreIndex.from_documents(
    policy_data,
    use_async=True,
).as_query_engine()
circular_1_policy_engine = VectorStoreIndex.from_documents(
    policy_circular_1,
    use_async=True,
).as_query_engine()
circular_2_policy_engine = VectorStoreIndex.from_documents(
    policy_circular_2,
    use_async=True,
).as_query_engine()

[Document(id_='a0bf74ed-20bf-46ca-a306-923ebeec7389', embedding=None, metadata={'page_label': '1', 'file_name': 'policy.pdf', 'file_path': 'data/policy.pdf', 'file_type': 'application/pdf', 'file_size': 445271, 'creation_date': '2024-04-16', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Password Policy  \n1. Purpose:  The purpose of this policy is to establish guidelines and requirements for the creation, \nmanagement, and protection of passwords to ensure the security and integrity of the organization's \ninformation systems.  \n2. Scope:  This policy applies to all employees, contractors, vendors, and any other individuals who have \naccess to the organization's information systems and resources.  \n 3. Password C

**********
Trace: index_construction
    |_node_parsing -> 0.000843 seconds
      |_chunking -> 0.000345 seconds
    |_embedding -> 0.0 seconds
    |_exception -> 0.0 seconds
**********


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Setup sub question query engine

In [ ]:

from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-0613")

In [ ]:
# adding the questionGenerator from query using the llm
from llama_index.question_gen.guidance import GuidanceQuestionGenerator
from llama_index.llms.openai import OpenAI as GuidanceOpenAI
question_gen = GuidanceQuestionGenerator.from_defaults(
    guidance_llm=llm, verbose=False
)

In [ ]:
from llama_index.core import get_response_synthesizer

summarizer = get_response_synthesizer(
    response_mode="refine",
    llm=llm,
    verbose=True,
    structured_answer_filtering=True,
)

In [ ]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=policy_data_engine,
        metadata=ToolMetadata(
            name="Policy_data",
            description="Company policy data, these rules and regulations are to be enforced given the circumstances.",
        ),
    ),
    QueryEngineTool(
        query_engine=circular_1_policy_engine,
        metadata=ToolMetadata(
            name="Policy_data_circular1",
            description="Changes in the policy data to better improve the execution of the rules",
        ),
    ),
    QueryEngineTool(
        query_engine=circular_2_policy_engine,
        metadata=ToolMetadata(
            name="policy_data_circular2",
            description="Changes in the policy data to better improve the execution of the rules ",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    # question_gen = question_gen,
    query_engine_tools=query_engine_tools,
    response_synthesizer= summarizer,
    use_async=False,
)

### Run queries

In [ ]:
query = '''
Subject: Urgent: Your Account has been Temporarily Locked

Dear [User],

We hope this email finds you well.

We regret to inform you that we have detected three unsuccessful login attempts on your account. As a result, for security reasons, your account has been temporarily locked.

If you have any further questions or concerns, please don't hesitate to reach out to us.

Thank you for your attention to this matter.

Best regards,

[Your Name]
[Your Position]
[Your Contact Information]
'''

In [ ]:
# query = '''
# Dear [Customer's Name],

# We wanted to inform you of a new policy regarding password retries for your account. Effective immediately, our system will restrict login attempts to three within a specified timeframe.

# This measure aims to bolster account security and prevent unauthorized access. Should you encounter any issues or have questions, please reach out to our support team.

# Thank you for your understanding and cooperation.

# Best regards,

# [Your Name]
# [Your Position]
# [Your Company]
# '''

In [ ]:
response = query_engine.query(
    f"Here is an email from an oraganization to employee of that organization:'{query}' that we are going to send out, does it align with latest Account logout policy?"
)

Generated 3 sub questions.
[Policy_data] Q: Is the latest Account logout policy being followed in the email communication?
[Policy_data] A: The latest Account Lockout policy states that after 3 unsuccessful login attempts, the account will be temporarily locked. Users can unlock their accounts through a secure and approved process.
[Policy_data_circular1] Q: Are there any recent changes in the policy data related to account lockouts?
[Policy_data_circular1] A: Yes, recent changes have been made to the policy data related to account lockouts. The adjustment now includes an automated process for account recovery, enabling users to securely unlock their accounts through a multi-factor authentication process. This change is intended to minimize work disruption and reduce administrative overhead for account recovery.
[policy_data_circular2] Q: Are there any recent changes in the policy data related to account lockouts?
[policy_data_circular2] A: Yes, there have been recent changes in the po

In [ ]:
"""Here is an email from an oraganization to employee of that organization:'{query}' that we are going to send out,
 does it align with policies? Also explain what is violating the policy. Also share the brief details of the rukes defined that are being violated."""

"Here is an email from an oraganization to employee of that organization:'{query}' that we are going to send out,\n does it align with policies? Also explain what is violating the policy. Also share the brief details of the rukes defined that are being violated."

In [ ]:
response.response

'Yes, the email aligns with the latest Account Lockout policy as it states that after 3 unsuccessful login attempts, the account will be temporarily locked.'

In [ ]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: Does the email align with the policy data?
Answer: The email should align with the policy data by emphasizing the importance of following secure password practices, including password creation guidelines, complexity requirements, expiration periods, reuse restrictions, account lockout procedures, multi-factor authentication, and secure password storage methods. It should also stress the significance of regular training, monitoring, auditing, and enforcement of the password policy to ensure compliance and enhance the security of the organization's information systems.
Sub Question 1: What is violating the policy in the email?
Answer: Sharing passwords with IT support personnel without approval and documentation is violating the policy in the email.
Sub Question 2: Brief details of the rules defined that are being violated in the email
Answer: Rules being violated in the email include sharing passwords with IT support personnel without approval, not following a secure pro